In [1]:
# Imoort Libraries
import os
import pandas as pd
import seaborn as sns
import env
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Mask for American Airlines

In [2]:
flights = wrangle.pull_airline_data('AA')

In [3]:
flights.columns

Index(['FL_DATE', 'OP_CARRIER', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'],
      dtype='object')

In [4]:
flights.OP_CARRIER.value_counts()

AA    7628937
Name: OP_CARRIER, dtype: int64

In [5]:
flights.head().T

,12005,12006,12007,12008,12009
FL_DATE,2009-01-01,2009-01-01,2009-01-01,2009-01-01,2009-01-01
OP_CARRIER,AA,AA,AA,AA,AA
CARRIER_DELAY,NaN,35.0,NaN,NaN,NaN
WEATHER_DELAY,NaN,0.0,NaN,NaN,NaN
NAS_DELAY,NaN,0.0,NaN,NaN,NaN
SECURITY_DELAY,NaN,0.0,NaN,NaN,NaN
LATE_AIRCRAFT_DELAY,NaN,0.0,NaN,NaN,NaN


In [ ]:
flights.duplicated().sum()

In [ ]:
flights.info()

In [ ]:
flights.describe()

In [ ]:
flights.shape

In [ ]:
# Check for missing values
missing_values = flights.isnull()
missing_values

In [ ]:
flights.isnull().sum()


### How are these columns related and how can I combine them

### Compare the columns by various measures

In [ ]:
#Fills in nulls as zero as null means no delay
flights.fillna(0, inplace=True)
#Makes FL_DATE column a datetime datatype
flights.FL_DATE = flights.FL_DATE.astype('datetime64')
#Makes FL_DATE the index
# flights = flights.set_index('FL_DATE')

In [ ]:
# Drop rows with any missing values
flights_dropped = flights.dropna()
flights_dropped


In [ ]:
flights_cat = flights
flights_cat['carrier'] = flights_cat.CARRIER_DELAY > 0

In [ ]:
flights_cat['weather'] = flights_cat.WEATHER_DELAY > 0

In [ ]:
flights_cat['nas'] = flights_cat.NAS_DELAY > 0

In [ ]:
flights_cat['security'] = flights_cat.SECURITY_DELAY > 0

In [ ]:
flights_cat['late_aircraft'] = flights_cat.LATE_AIRCRAFT_DELAY > 0

In [ ]:
flights_cat.drop(columns=['OP_CARRIER', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'])

In [ ]:
flights.info()

### At first glance it seems weird that weather does not have as many rows as the rest.  As a pilot I thought that weather dominates the delays so I wondered why this is the case.

### It seems that the NAS data includes non extreme weater as does carrier and late aircraft delay.

In [ ]:
dep = pd.crosstab(flights_cat.FL_DATE, [flights_cat.carrier, flights_cat.weather, flights_cat.nas, flights_cat.security, flights_cat.late_aircraft])

### Chi squared
* Null Hypothesis:  These delay types are independent of each other
* Alternative Hypothesis:  These types are dependent on each other
* Alpha < 0.05

In [ ]:
dep

In [ ]:
chi2, p, degf, expected = stats.chi2_contingency(dep)

In [ ]:
chi2, p

#### There is sufficient evidence to reject the null.  They are dependent on each other.

#### This suggests that there may bee some sort of overlap.  A quick look at the NAS delays shows that minor weather phenomena show up in the NAS as opposed to weather.  Also a weather delay on one leg of an airplanes trip seems to turn into a carrier delay on the next.

### A look at histograms

In [ ]:
plt.hist(flights.CARRIER_DELAY, bins=100)

In [ ]:
plt.hist(flights.WEATHER_DELAY, bins=100)

In [ ]:
plt.hist(flights.NAS_DELAY, bins=100)

In [ ]:
plt.hist(flights.SECURITY_DELAY, bins=100)

In [ ]:
plt.hist(flights.LATE_AIRCRAFT_DELAY, bins=100)

#### Skewed makes a whole lot of sense.  Nothing unexpected here.

### THERE IS sufficient evidence that I can combine these columns in feature engineering.

### Resample my data by daily mean delays

In [ ]:
flights_daily_mean = wrangle.clean_flight_data_for_average_daily_delay(flights)

In [ ]:
flights_daily_mean.sort_index()

In [ ]:
flights_daily_mean.isna().sum()

### Find the best interval to define average delay by to optimize reducing noise while keeping good correlation results

In [ ]:
def plot_average_by_interval():
    
    resample_dict = {'D':'Daily', 'W':'Weekly', '2W':'FORTNIGHTLY', 'M':'Monthly', '3M':'Quarterly', '6M':'Semi-Annually', 'Y':'Yearly'}

    for _ in resample_dict:
        plt.figure(figsize=(10,6))

        flights_daily_mean.resample(_).mean().average_delay.plot(label='daily')
        # flights_daily_mean.resample('W').mean().average_delay.plot(label='weekly')
        # flights_daily_mean.resample('M').mean().average_delay.plot(label='monthly')
        # flights_daily_mean.resample('Y').mean().average_delay.plot(label='yearly')


        plt.title(f'{resample_dict[_][0:]} Average Delay')
        plt.legend()
        plt.show()

        pd.plotting.autocorrelation_plot(flights_daily_mean.average_delay.resample(_).mean())
        plt.show()

plot_average_by_interval()

### Plot the best lag plot based on the airline.  The best will always be 365

In [ ]:
def plot_best_lag_plot():

    #Plots the best correlated lag
    pd.plotting.lag_plot(flights_daily_mean.resample('W').mean(), lag=365)
    plt.title('lag plot');

plot_best_lag_plot()

### Split the data by year

In [ ]:
train = flights_daily_mean.iloc[:2016] 
test = flights_daily_mean.iloc[2016:] 
train.loc['2009-09-30':]

In [ ]:
test

### Plot the seasonal decompositions for the various time periods

In [ ]:
y = train.average_delay.resample('d').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('w').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
def seasonal_decomposition_plot(duration):
    y = train.average_delay.resample(duration).mean()

    result = sm.tsa.seasonal_decompose(y)

    decomposition = pd.DataFrame({
        'y': result.observed,
        'trend': result.trend,
        'seasonal': result.seasonal,
        'resid': result.resid
    })

    decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
    decomposition[['trend_centered', 'seasonal', 'resid']].plot();
    
seasonal_decomposition_plot('2w')

In [ ]:
y = train.average_delay.resample('m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('3m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('6m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('y').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

### Once again the best period to resample by seems to be 2 weeks or a fortnight